<a href="https://colab.research.google.com/github/Chuck2Win/chatbot_project/blob/main/kogpt2_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. GPU 확인


In [ ]:
! nvidia-smi

Mon Nov 16 23:04:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 2. Google drive 연동


In [2]:
import os
from google.colab import drive
# google mount
drive.mount('/content/gdrive')
os.chdir('./gdrive/My Drive/chatbot/KoGPT2')

Mounted at /content/gdrive


# 2. 1 필요 패키지 설치
transformers

In [3]:
! pip3 install transformers

     |████████████████████████████████| 1.3MB 6.0MB/s 
     |████████████████████████████████| 890kB 41.8MB/s 
     |████████████████████████████████| 2.9MB 45.0MB/s 
     |████████████████████████████████| 1.1MB 53.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1707c6cb358c36a188b5a5857154bcb32f4adc4c0a63c679a5e9f49463eeb301
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import torch
import torch.nn as nn
import os
import torchtext
import time
import datetime
from tqdm import tqdm
import transformers

# 3. Data 불러오기
1. Chatbot_data	일상 챗봇 학습용 문답 페어 11,876건
일상(0) / 이별,부정(1) / 사랑,긍정(2) 라벨 부여 

# 3.1 데이터 전처리

In [5]:
# model, tokenizer
model=transformers.GPT2LMHeadModel.from_pretrained('taeminlee/kogpt2')
tokenizer=transformers.GPT2TokenizerFast.from_pretrained('taeminlee/kogpt2')

In [6]:
# special token 추가
special_tokens = {"cls_token":"<cls>"}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))
print(tokenizer.all_special_tokens)

['<s>', '</s>', '<unk>', '<pad>', '<cls>']


# model load

In [ ]:
class Config(dict):
    __setattr__=dict.__setitem__
    __getattr__=dict.__getitem__

In [ ]:
config=Config({'seq_len':64,'bos_idx':tokenizer.eos_token_id,'eos_idx':tokenizer.bos_token_id,'pad_idx':tokenizer.pad_token_id,'cls_idx':tokenizer.encode('<cls>')[0],'n_vocab':tokenizer.vocab_size+1, 'batch_size':2})

In [ ]:
# bos token, eos token 붙이기
class make_data_set:
    def __init__(self,config):
        '''
        data : data frame
        data - Q, A 로 구성 
        '''
        data=pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv',sep=',',header=0)
        data['Q']='<s>'+data['Q']+'<cls>' #  start token, cls token add 
        data['A']='<s>'+data['A'] # start token, end token add
        
        data['Q_input_id']=data['Q'].apply(lambda i : tokenizer.encode(i))
        data['A_input_id']=data['A'].apply(lambda i : tokenizer.encode(i))
        
        data['attention_mask']=None
        data['token_id']=None
        data['cls_position']=None
        Max = config.seq_len
        for i in data.index:
            if len(data.Q_input_id[i])+len(data.A_input_id[i])>Max:
                data.Q_input_id[i]=data.Q_input_id[i][-(Max//2):]
                data.A_input_id[i]=data.A_input_id[i][:(Max-(Max//2))]
            data['attention_mask'][i] = [1] * (len(data.Q_input_id[i])+len(data.A_input_id[i]))+[0] * (Max - (len(data.Q_input_id[i])+len(data.A_input_id[i])))
            data['token_id'][i] = [0] * len(data.Q_input_id[i]) + [1] * (Max-len(data.Q_input_id[i]))
            data['cls_position'][i] = len(data.Q_input_id[i])-1
        data['input_id']=data['Q_input_id']+data['A_input_id']
        data['input_id']=data['input_id'].apply(lambda j : j+(Max-len(j))*[tokenizer.pad_token_id])
        data['lm_label']=data['Q_input_id'].apply(lambda i : len(i) * [tokenizer.pad_token_id])+data['A_input_id'].apply(lambda i : i[1:]+[tokenizer.eos_token_id])
        data['lm_label']=data['lm_label'].apply(lambda j : j+(Max-len(j))*[tokenizer.pad_token_id])
        self.data=data

    def return_data_set(self):
        result=self.data.loc[:,['input_id','attention_mask','token_id','cls_position','lm_label','label']]
        return result
# gpt2 관련 input들 처리하기
# index of words - tokenized되고 encode 된 id들
# token type ids - [0] : question [1] : answer 
# attention mask - [1] : padding 안 된 부분 [0] : padding 된 부분
# label : Q 
# cls position : cls의 위치 (index)

In [ ]:
make_data=make_data_set(config)
dataset=make_data.return_data_set()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
dataset.head()

,input_id,attention_mask,token_id,cls_position,lm_label,label
0,"[0, 1494, 47812, 50000, 0, 49517, 48594, 48594...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,"[3, 3, 3, 3, 49517, 48594, 48594, 47440, 1, 3,...",0
1,"[0, 47456, 49517, 50000, 0, 49517, 47440, 3, 3...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,"[3, 3, 3, 3, 49517, 47440, 1, 3, 3, 3, 3, 3, 3...",0
2,"[0, 47495, 48594, 47514, 48594, 50000, 0, 4951...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5,"[3, 3, 3, 3, 3, 3, 49517, 47440, 1, 3, 3, 3, 3...",0
3,"[0, 47495, 48594, 47514, 48594, 50000, 0, 4951...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5,"[3, 3, 3, 3, 3, 3, 49517, 47440, 1, 3, 3, 3, 3...",0
4,"[0, 47754, 9003, 49517, 50000, 0, 48594, 49517...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4,"[3, 3, 3, 3, 3, 48594, 49517, 47440, 1, 3, 3, ...",0


### 데이터 저장

In [ ]:
dataset.to_csv('./data.csv')

### train test split
test는 0.2 비율

In [1]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(dataset,test_size=0.2)

NameError: ignored

### DataLoader

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
train_dataset=TensorDataset(torch.LongTensor(train.input_id.values.tolist()),torch.LongTensor(train.attention_mask.values.tolist()),torch.LongTensor(train.token_id.values.tolist()),torch.LongTensor(train.cls_position.values.tolist()),torch.LongTensor(train.lm_label.values.tolist()),torch.LongTensor(train.label.values.tolist()))
test_dataset=TensorDataset(torch.LongTensor(test.input_id.values.tolist()),torch.LongTensor(test.attention_mask.values.tolist()),torch.LongTensor(test.token_id.values.tolist()),torch.LongTensor(test.cls_position.values.tolist()),torch.LongTensor(test.lm_label.values.tolist()),torch.LongTensor(test.label.values.tolist()))

In [ ]:
# train loader, test loader
train_loader=DataLoader(train_dataset,batch_size=2,drop_last=True)
test_dataset=DataLoader(test_dataset,batch_size=2,drop_last=True)

In [ ]:
class my_kogpt2_model(nn.Module):
    def __init__(self, config, kogpt2):
        super().__init__()
        self.config = config
        self.model = kogpt2
        self.classification = nn.Linear(self.config.n_vocab, 3)
    
    def forward(self,data):
        input_ids=data[0]
        attention_mask=data[1]
        token_type_ids=data[2]
        cls_position=data[3] # batch size 
        lm_label=data[4]
        label=data[-1]
        output = self.model.forward(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,return_dict=True)
        lm_output = output.logits # batch size, seq len, n_vocab -> batch size, n_vocab
        classification_output = torch.vstack([self.classification.forward(lm_output[_,cls_position[_]]) for _ in range(self.config.batch_size)])
        return lm_output, classification_output

## 4. 학습

In [ ]:
import time,datetime
def Eplased(dt):
    d=int(round(dt))
    return str(datetime.timedelta(seconds=d))

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
epochs = 50         # Num of Epoch
learning_rate = 1e-5
Model = my_kogpt2_model(config, model)
Model.to(device)
optimizer = transformers.AdamW(model.parameters(),lr=learning_rate) 
criterion1 = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id,reduction='sum') # language modeling
criterion2 = nn.CrossEntropyLoss() # classification

In [ ]:
model.train()
start_time = time.time()
for epoch in tqdm(range(1,epochs+1),desc='epoch',mininterval=3600):
    total_loss_1=0.
    total_loss_2=0.
    for step, data in tqdm(enumerate(train_loader),desc='step',mininterval=600):
        optimizer.zero_grad()
        data = tuple(i.to(device) for i in data)
        a,b=Model.forward(data)        
        loss1=criterion1(a.reshape(-1,50001),data[4].reshape(-1))
        loss2=criterion2(b,data[-1])
        loss=loss1/config.batch_size+loss2/config.batch.size
        loss.backward()
        total_loss_1+=(loss1/config.batch_size).item()
        total_loss_2+=(loss2).item()
        optimizer.step()
    print('epoch : %d'%epoch)
    print('total loss 1 : %.3f'%(total_loss_1/len(train_loader)))
    print('total loss 2 : %.3f'%(total_loss_2/len(train_loader)))
    print('Eplased Time : %s'%(Eplased(time.time()-start_time)))

epoch:   0%|          | 0/50 [00:00<?, ?it/s]
step: 4729it [04:44, 16.65it/s]

step: 0it [00:00, ?it/s]

epoch : 1
total loss 1 : 5.799
total loss 2 : 0.965
Eplased Time : 0:04:44


step: 4729it [04:43, 16.66it/s]

step: 0it [00:00, ?it/s]

epoch : 2
total loss 1 : 5.779
total loss 2 : 0.944
Eplased Time : 0:09:28


step: 4729it [04:43, 16.68it/s]

step: 0it [00:00, ?it/s]

...

epoch : 48
total loss 1 : 4.450
total loss 2 : 0.849
Eplased Time : 3:46:24


step: 4729it [04:45, 16.58it/s]

step: 0it [00:00, ?it/s]

epoch : 49
total loss 1 : 4.426
total loss 2 : 0.849
Eplased Time : 3:51:09


step: 4729it [04:44, 16.64it/s]
epoch: 100%|██████████| 50/50 [3:55:53<00:00, 283.06s/it]

epoch : 50
total loss 1 : 4.400
total loss 2 : 0.845
Eplased Time : 3:55:53


In [ ]:
torch.save(model.state_dict(),'./chatbot_transformer')